In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd

from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from matplotlib import pyplot
import rasterio.mask
import fiona
import json

In [2]:
OSD_df = pd.read_parquet("generated_files/bio_variables_dataframe_occurence.parquet")

# Creating Geometry
OSD_df['geometry'] = list(zip(OSD_df["Longitude"], OSD_df["Latitude"]))
OSD_df['geometry'] = OSD_df["geometry"].apply(Point)

# Create the geodataframe
OSD_geoframe = gpd.GeoDataFrame(
    OSD_df,
    crs = {'init': 'epsg:4326'},
    geometry = OSD_df['geometry']
)
OSD_geoframe = OSD_geoframe.to_crs("EPSG:4326")
OSD_geoframe.reset_index(drop=True, inplace = True)

OSD_geoframe['Year'] = OSD_geoframe['Data'].dt.year
OSD_df = pd.DataFrame(OSD_geoframe)

c:\Users\kenji\dev\web-scraping-images-vinacea\web-scraping-images-vinacea\web-scraping-venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [3]:
base_dataframe = pd.read_parquet("generated_files/bio_variables_dataframe_base.parquet")

In [4]:
# Creating Geometry
base_dataframe['geometry'] = list(zip(base_dataframe["Longitude"], base_dataframe["Latitude"]))
base_dataframe['geometry'] = base_dataframe["geometry"].apply(Point)

# Create the geodataframe
base_dataframe = gpd.GeoDataFrame(
    base_dataframe,
    crs = {'init': 'epsg:4326'},
    geometry = base_dataframe['geometry']
)
base_dataframe = base_dataframe.to_crs("EPSG:4326")
base_dataframe.reset_index(drop=True, inplace = True)

base_dataframe = pd.DataFrame(base_dataframe)

c:\Users\kenji\dev\web-scraping-images-vinacea\web-scraping-images-vinacea\web-scraping-venv\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [5]:
img_config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'species_distribution',
    'scale':4 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

In [6]:
# Source: IBGE:  bcim_2016_21_11_2018
INFOS_UFS = gpd.read_file("assets/bcim_2016_21_11_2018.gpkg", layer = 'lim_unidade_federacao_a')[['sigla','geometry']].rename(columns={'sigla':'stateProvince'})
INFOS_UFS.to_file("assets/UFS_JSON", driver = "GeoJSON")
with open("assets/UFS_JSON") as geofile:
    j_file = json.load(geofile)

In [7]:
fig = px.scatter_mapbox(
    OSD_df,
    lat="Latitude",
    lon="Longitude",
    color_discrete_sequence=[px.colors.qualitative.Plotly[1]],
    color="01_annual_mean_temp",
    zoom=4,
    height=500,
    opacity=0.9
)
"""
fig_graph = px.scatter_mapbox(
    OSD_df,
    lat="Latitude",
    lon="Longitude",
    zoom=4,
    opacity=1
).data[0]
fig.add_trace(fig_graph)
"""
fig.update_layout(autosize=True,margin={"r": 0, "t": 0, "l": 0, "b": 0},mapbox_style="carto-positron")
fig.update_geos(
    visible=False,
    resolution=50,
    lataxis_gridcolor="red",
    lataxis_showgrid=True,
    lataxis_dtick=15,
    lonaxis_showgrid=True,
    lonaxis_dtick=15,
)
#fig.write_image('figure.png', scale=2)
fig.show(config=img_config)